In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import time
import tensorflow as tf
import p8_util
import p8_util_config

Using TensorFlow backend.


In [3]:
import p8_util
datadir = './data'

filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, feature_shape = p8_util.load_dataset(filename_dataset,is_label_encoded=True)

print("\nDataset : {}".format((x_train.shape, x_test.shape, y_train.shape, y_test.shape, nClasses, feature_shape)))

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3)

Dataset : ((414, 224, 224, 3), (47, 224, 224, 3), (414,), (47,), 3, (224, 224, 3))


In [4]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = True

In [5]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (31361)kill: (31367)

#### Increase of train dataset against test dataset.

In [6]:
OUTPUT_DIR

'./tmp/baseline'

In [30]:
import p8_util
print("Input output dir= {}".format(OUTPUT_DIR))
oNNAdaNetBuilder = p8_util.create_nn_builder(feature_shape,OUTPUT_DIR)
oNNAdaNetBuilder.show()

Input output dir= ./tmp/baseline


Adanet outputdir     : ............................ ./tmp/baseline/CNN
Adanet output log    : ............................ ./tmp/baseline/CNN/CNN
Adanet iterations    : ............................ 10
NN type              : ............................ CNN
Features shape       : ............................ (224, 224, 3)
Number of layers     : ............................ None
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 0.001
Weights initializer  : ............................ xavier
Batch normalization  : ............................ True
Learn mixture weights: ............................ True


CNN seed             : ............................ 42
Conv. Kernel size    : ............................ (5, 5)
Conv. layers         : ............................ None
Dense layers    

In [18]:
oNNAdaNetBuilder._last_layer

In [19]:
p8_util_config.dict_adanet_config

{'adanet_feature_columns': None,
 'adanet_feature_shape': None,
 'adanet_tf_head': None,
 'adanet_lambda': 0.001,
 'adanet_is_learn_mixture_weights': True,
 'adanet_initial_num_layers': 1,
 'adanet_num_layers': None,
 'adanet_nn_candidate': 2,
 'adanet_output_dir': './tmp/adanet',
 'adanet_nn_layer_config': {'nn_type': 'RNN',
  'nn_dropout_rate': 0.0,
  'nn_batch_norm': True,
  'nn_logit_dimension': 3,
  'nn_optimizer': <tensorflow.python.training.adam.AdamOptimizer at 0x7f0b96400ba8>,
  'nn_seed': 42,
  'nn_initializer_name': 'truncated_normal',
  'nn_layer_config': {'rnn_layer_num': 2,
   'rnn_hidden_units': 16,
   'rnn_timesteps': 224,
   'rnn_cell_type': 'SGRU'}}}

In [20]:
import p8_util

batch_test = 20
input_fn_param={'num_epochs':p8_util_config.NUM_EPOCHS,\
                'batch_size':p8_util_config.BATCH_SIZE,\
                'feature_shape': oNNAdaNetBuilder.feature_shape,\
               }
train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        max_steps=p8_util_config.MAX_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        steps=None,\
        throttle_secs=1)

In [21]:
params = {'net_builder': oNNAdaNetBuilder}
    
classifier = tf.estimator.Estimator(model_fn=p8_util.custom_model_fn, params=params, config=oNNAdaNetBuilder.classifier_config)

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/RNN/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0b962d16a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0513 14:57:25.972729 139689580656448 estimator.py:201] Using config: {'_model_dir': './tmp/baseline/RNN/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0b962d16a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
import p8_util

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.


I0513 14:57:26.618523 139689580656448 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


I0513 14:57:26.620570 139689580656448 training.py:610] Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.


...

I0513 15:00:40.162508 139689580656448 estimator.py:1979] Saving dict for global step 4: RNN_Eval_accuracy = 0.33816424, global_step = 4, loss = 1.0985215


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4: ./tmp/baseline/RNN/SGRU/model.ckpt-4


I0513 15:00:41.417714 139689580656448 estimator.py:2039] Saving 'checkpoint_path' summary for global step 4: ./tmp/baseline/RNN/SGRU/model.ckpt-4


INFO:tensorflow:Loss for final step: 1.091985.


I0513 15:00:41.955684 139689580656448 estimator.py:359] Loss for final step: 1.091985.


In [23]:
print(oNNAdaNetBuilder._last_layer)
oNNAdaNetBuilder.show()

None


Adanet outputdir     : ............................ ./tmp/baseline/RNN
Adanet output log    : ............................ ./tmp/baseline/RNN/SGRU
NN type              : ............................ RNN
Features shape       : ............................ (224, 224, 3)
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 0.001
Weights initializer  : ............................ truncated_normal
Batch normalization  : ............................ True
Learn mixture weights: ............................ True


Cell type            : ............................ SGRU
Hidden units         : ............................ 16
Stacked cells        : ............................ 2
Time steps           : ............................ 224


In [14]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))
print("------------------------------------------------\n")



------------------------------------------------
Time (sec) 3.7172183990478516
DNN_EVAL_ACCURACY: 0.33816424012184143
LOSS: 19.677350997924805
GLOBAL_STEP: 4
------------------------------------------------



In [15]:
oNNAdaNetBuilder.output_dir_log

'./tmp/baseline/DNN/DNN'

In [16]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(oNNAdaNetBuilder.output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')